In [1]:
# Import necessary packages
import pandas as pd
import numpy as np
import os,sys


In [2]:
# Check current working directory
os.getcwd()

'/Users/yeonjoo/Downloads/Unilateral_foot'

In [116]:
# Change directory if necessary
#newdirectory = input("Path: ")
#os.chdir(newdirectory)
#os.getcwd()

In [3]:
# File import by filename
Subject = input("Subject: ")
Trial = input("Trial: ")
filename = Subject + "Trial"+ Trial

df = pd.read_csv(filename +'.csv')
df.head()

Subject: B01
Trial: 01B


,LHEEX,LHEEY,LHEEZ,LTOEX,LTOEY,LTOEZ,RHEEX,RHEEY,RHEEZ,RTOEX,RTOEY,RTOEZ
0,213.487488,237.421875,448.451904,152.920929,211.413589,268.553162,-42.172558,291.961273,233.400925,-237.794388,320.854614,214.888504
1,206.448959,237.571869,448.444214,143.744415,209.842484,269.410187,-36.156570,292.136383,233.808090,-231.766464,321.187836,215.134430
2,197.450485,237.683365,447.939270,131.803299,207.946152,270.171112,-28.587440,292.350067,234.323608,-224.168304,321.603699,215.438858
3,186.790955,237.709122,446.701294,117.349838,205.832184,270.662994,-19.769789,292.589478,234.928970,-215.298050,322.083527,215.786713
4,174.854614,237.606277,444.523987,100.744469,203.620560,270.738037,-10.068398,292.839020,235.602142,-205.517883,322.605591,216.160553


In [628]:
# Use Only When first rows are "0" =====> Need to code "IF"
df.drop([0,1], inplace=True)
df = df.reset_index(drop=True)
df.head()

,LHEEX,LHEEY,LHEEZ,LTOEX,LTOEY,LTOEZ,RHEEX,RHEEY,RHEEZ,RTOEX,RTOEY,RTOEZ
0,403.644684,-910.215576,259.211395,348.533875,-1074.252808,226.482574,216.669968,-1239.818359,260.551208,147.025513,-1410.347290,244.652054
1,403.669647,-905.067993,259.794403,348.768158,-1068.992798,226.475479,217.869171,-1240.009033,259.320648,147.542450,-1410.262451,242.443832
2,403.693420,-899.431030,260.531158,349.034241,-1063.204834,226.461304,219.183731,-1239.246094,258.129730,148.082626,-1409.170166,239.877090
3,403.718964,-893.523071,261.444885,349.306885,-1057.097412,226.441513,220.475861,-1237.392456,257.077484,148.597427,-1406.970825,237.175125
4,403.749298,-887.527039,262.568176,349.560516,-1050.841187,226.415161,221.618820,-1234.469971,256.237274,149.042221,-1403.720337,234.601379


In [5]:
# Calculate Heel Marker Velocity
col_names = ['LHEEX','RHEEX', 'LTOEX', 'RTOEX']
name_rows_dict1 = {}

for col in col_names:
    vel_rows = []
    for idx, value in enumerate(df[col]):
        if idx == 0:
            vel_rows.append(0)
        else:
            vel_row = (df.loc[idx,col] - df.loc[idx-1,col])*100
            vel_rows.append(vel_row)
    name_rows_dict1[col] = vel_rows

vel_df = pd.DataFrame(name_rows_dict1)
vel_df = vel_df.drop(0)
vel_df = vel_df.reset_index(drop=True)
vel_df.head(10)

,LHEEX,RHEEX,LTOEX,RTOEX
0,-703.8529,601.5988,-917.6514,602.7924
1,-899.8474,756.9130,-1194.1116,759.8160
2,-1065.9530,881.7651,-1445.3461,887.0254
3,-1193.6341,970.1391,-1660.5369,978.0167
4,-1285.5835,1025.0387,-1838.1722,1035.1898
5,-1350.7934,1053.7876,-1980.9089,1065.1504
6,-1400.0397,1064.9562,-2093.3484,1075.6684
7,-1442.8436,1066.0969,-2181.0028,1073.9746
8,-1486.2793,1062.1841,-2249.8067,1066.1911
9,-1535.0281,1055.6671,-2305.3649,1056.9855


In [6]:
# Identify Left Heel Strike
# by Left Heel marker velocity changes its direction
name_rows_dict2 = {}

lstrike_rows = []
for idx, value in enumerate(vel_df['LHEEX']):
    if idx == 0:
        idx = idx + 1
    else:
        if vel_df.loc[idx, 'LHEEX'] < 0:
            idx = idx + 1 
        else:
            if vel_df.loc[idx - 1, 'LHEEX'] > 0:
                idx = idx + 1 
            else:
                lstrike_rows.append(idx - 1)
            
name_rows_dict2['LHSidx'] = lstrike_rows

lstrike_df = pd.DataFrame(name_rows_dict2)
lstrike_df.head()

,LHSidx
0,30
1,139
2,247
3,357
4,467


In [9]:
# Identify left side repeated step (first interval default 50)

lefts = []
for idx in lstrike_df.index:
    if idx == 0:
        lefts.append(55)
    else:
        left = (lstrike_df.loc[idx, 'LHSidx']- lstrike_df.loc[idx - 1, 'LHSidx'])
        lefts.append(left)

lstrike_df['lHSinterval'] = lefts
lstrike_df.head(), lstrike_df['lHSinterval'].unique(), lstrike_df.count()

(   LHSidx  lHSinterval
 0      30           55
 1     139          109
 2     247          108
 3     357          110
 4     467          110,
 array([ 55, 109, 108, 110, 112, 114, 113, 111, 107, 115, 106, 104, 105,
        103, 102, 117, 116]),
 LHSidx         279
 lHSinterval    279
 dtype: int64)

In [9]:
#Drop left side repeated step
for idx, row in lstrike_df.iterrows():
    if row['lHSinterval'] < 55:
        lstrike_df.drop(idx, inplace=True)


lstrike_df.head(), lstrike_df['lHSinterval'].unique(), lstrike_df.count()

(   LHSidx  lHSinterval
 0      87           55
 1     186           99
 2     287          101
 3     388          101
 4     487           99,
 array([ 55,  99, 101, 102, 107, 106, 103, 104, 105, 100,  95, 109, 132,
         96,  94,  98,  97,  92,  91,  93,  89,  90, 108,  70]),
 LHSidx         301
 lHSinterval    301
 dtype: int64)

In [790]:
#lstrike_df.index[(lstrike_df['lHSinterval']== 76)]

Int64Index([181], dtype='int64')

In [791]:
#lstrike_df.iloc[181]

LHSidx         25048
lHSinterval       70
Name: 182, dtype: int64

In [792]:
#lstrike_df1 = lstrike_df.drop(181)

In [794]:
#lstrike_df1.iloc[175:185]

,LHSidx,lHSinterval
176,24348,147
177,24481,133
178,24626,145
179,24764,138
180,24902,138
182,25048,70
183,25185,137
184,25318,133
185,25452,134
186,25585,133


In [696]:
#lstrike_df.index[(lstrike_df['lHSinterval']== 83)]

Int64Index([172], dtype='int64')

In [657]:
#lstrike_df.iloc[170:175]

,LHSidx,lHSinterval
170,23658,141
171,23787,129
172,23841,54
173,23924,83
174,24064,140


In [697]:
#rstrike_df.iloc[170:175]

,RHSidx,rHSinterval,RHSval
170,23592,136,-1287.027588
171,23726,134,-1306.150635
172,23854,128,-1297.627930
173,23998,144,-1260.252075
174,24132,134,-1287.034058


In [10]:
# Reset the index
lstrike_df = lstrike_df.reset_index(drop=True)
lstrike_df.head()

,LHSidx,lHSinterval
0,87,55
1,186,99
2,287,101
3,388,101
4,487,99


In [11]:
# Get trajectory for left heelstrike
LHSval = pd.DataFrame(df.loc[lstrike_df['LHSidx'], 'LHEEY'])
LHSval_reidx = LHSval.reset_index().drop(columns=['index'])

lstrike_df['LHSval'] = LHSval_reidx['LHEEY']
lstrike_df.head()

,LHSidx,lHSinterval,LHSval
0,87,55,105.401031
1,186,99,199.771591
2,287,101,131.252289
3,388,101,151.066971
4,487,99,139.325607


In [12]:
#NEW!!!! #Identify right heel strike
name_rows_dict3 = {}

rstrike_rows = []
for idx, value in enumerate(vel_df['RHEEX']):
    if idx == 0:
        idx = idx + 1
    else:
        if vel_df.loc[idx, 'RHEEX'] < 0:
            idx = idx + 1 
        else:
            if vel_df.loc[idx - 1, 'RHEEX'] > 0:
                idx = idx + 1 
            else:
                rstrike_rows.append(idx - 1)
            
name_rows_dict3['RHSidx'] = rstrike_rows

rstrike_df = pd.DataFrame(name_rows_dict3)
rstrike_df.head()

,RHSidx
0,35
1,136
2,234
3,335
4,435


In [13]:
# Identify right side repeated step (first interval default 50)

rights = []
for idx in rstrike_df.index:
    if idx == 0:
        rights.append(50)
    else:
        right = (rstrike_df.loc[idx, 'RHSidx']- rstrike_df.loc[idx - 1, 'RHSidx'])
        rights.append(right)

rstrike_df['rHSinterval'] = rights
rstrike_df.head(), rstrike_df['rHSinterval'].unique(),  rstrike_df.count()

(   RHSidx  rHSinterval
 0      35           50
 1     136          101
 2     234           98
 3     335          101
 4     435          100,
 array([ 50, 101,  98, 100, 102, 103, 105, 106, 104, 108, 107,  93,  45,
         97,  99,  91,  96,  90,  95,  92,  94,  89,  87,  77,  76, 109]),
 RHSidx         303
 rHSinterval    303
 dtype: int64)

In [14]:
rstrike_df.index[(rstrike_df['rHSinterval']== 45)]

Int64Index([46], dtype='int64')

In [16]:
lstrike_df.iloc[40:50]

,LHSidx,lHSinterval,LHSval
40,4192,101,191.119904
41,4301,109,174.553864
42,4402,101,179.598114
43,4508,106,215.778732
44,4611,103,207.412552
45,4743,132,191.657104
46,4847,104,206.956161
47,4950,103,166.713013
48,5055,105,169.356125
49,5161,106,144.526810


In [15]:
rstrike_df.iloc[40:50]

,RHSidx,rHSinterval
40,4140,102
41,4245,105
42,4351,106
43,4455,104
44,4557,102
45,4650,93
46,4695,45
47,4792,97
48,4897,105
49,5002,105


In [17]:
#Drop right side repeated step
for idx, row in rstrike_df.iterrows():
    if row['rHSinterval'] < 50:
        rstrike_df.drop(idx, inplace=True)

rstrike_df.head(), rstrike_df['rHSinterval'].unique(), rstrike_df.count()

(   RHSidx  rHSinterval
 0      35           50
 1     136          101
 2     234           98
 3     335          101
 4     435          100,
 array([ 50, 101,  98, 100, 102, 103, 105, 106, 104, 108, 107,  93,  97,
         99,  91,  96,  90,  95,  92,  94,  89,  87,  77,  76, 109]),
 RHSidx         302
 rHSinterval    302
 dtype: int64)

In [18]:
# Reset the index
rstrike_df = rstrike_df.reset_index(drop=True)
rstrike_df.head()

,RHSidx,rHSinterval
0,35,50
1,136,101
2,234,98
3,335,101
4,435,100


In [19]:
RHSval = pd.DataFrame(df.loc[rstrike_df['RHSidx'], 'RHEEY'])
RHSval_reidx = RHSval.reset_index().drop(columns=['index'])

rstrike_df['RHSval'] = RHSval_reidx['RHEEY']
rstrike_df.head()

,RHSidx,rHSinterval,RHSval
0,35,50,267.845825
1,136,101,279.315369
2,234,98,265.806152
3,335,101,229.240982
4,435,100,244.880814


In [20]:
strike_df = pd.merge(lstrike_df, rstrike_df, left_index = True, right_index = True)
strike_df.head()

,LHSidx,lHSinterval,LHSval,RHSidx,rHSinterval,RHSval
0,87,55,105.401031,35,50,267.845825
1,186,99,199.771591,136,101,279.315369
2,287,101,131.252289,234,98,265.806152
3,388,101,151.066971,335,101,229.240982
4,487,99,139.325607,435,100,244.880814


In [878]:
# Calculate step time


Rsteptime = []
Lsteptime = []

for idx in strike_df.index:
    if strike_df.loc[0, 'LHSidx'] < strike_df.loc[0, 'RHSidx']:
        if idx == 0:
            Rstep_time = strike_df.loc[idx, 'RHSidx'] - strike_df.loc[idx, 'LHSidx']
            Rsteptime.append(Rstep_time)
            Lsteptime.append(0)
        else:
            Rstep_time = strike_df.loc[idx, 'RHSidx'] - strike_df.loc[idx, 'LHSidx']
            Rsteptime.append(Rstep_time)
            Lstep_time = strike_df.loc[idx, 'LHSidx'] - strike_df.loc[idx - 1,'RHSidx']
            Lsteptime.append(Lstep_time)
    else:
        if idx ==0:
            Rsteptime.append(0)
            Lstep_time = strike_df.loc[idx, 'LHSidx'] - strike_df.loc[idx,'RHSidx']
            Lsteptime.append(Lstep_time)
        else:
            Rstep_time = strike_df.loc[idx, 'RHSidx'] - strike_df.loc[idx - 1, 'LHSidx']
            Rsteptime.append(Rstep_time)
            Lstep_time = strike_df.loc[idx, 'LHSidx'] - strike_df.loc[idx,'RHSidx']
            Lsteptime.append(Lstep_time)
        

strike_df['Rstep_time'] = Rsteptime
strike_df['Lstep_time'] = Lsteptime

strike_df.iloc[50:100]

,LHSidx,lHSinterval,LHSval,RHSidx,rHSinterval,RHSval,Rstep_time,Lstep_time,Rstep_length,Lstep_length
50,7152,141,-1266.496582,7221,144,-1277.430298,69,75,361.666284,406.285645
51,7293,141,-1285.066162,7367,146,-1293.572632,74,72,391.093530,381.164136
52,7433,140,-1330.547729,7501,134,-1291.425537,68,66,406.322192,319.424903
53,7576,143,-1270.454102,7642,141,-1272.707764,66,75,354.146338,425.971435
54,7712,136,-1291.017700,7780,138,-1264.387451,68,70,393.830249,359.690064
55,7849,137,-1240.423462,7921,141,-1295.171509,72,69,334.051953,396.563989
56,8006,157,-1314.625732,8078,157,-1270.656372,72,85,432.769360,439.545777
57,8153,147,-1288.958496,8226,148,-1258.318115,73,75,424.840381,386.697876
58,8309,156,-1307.808838,8386,160,-1286.879272,77,83,436.729566,398.709277
59,8465,156,-1299.889526,8530,144,-1312.237183,65,79,338.652343,413.589746


In [21]:
# Calculate step time 2
Rsteptime = []
Lsteptime = []

for idx in strike_df.index:
    if idx == 0:
        if strike_df.loc[0, 'LHSidx'] < strike_df.loc[0, 'RHSidx']:
            Rstep_time = strike_df.loc[idx, 'RHSidx'] - strike_df.loc[idx, 'LHSidx']
            Rsteptime.append(Rstep_time)
            Lsteptime.append(0)
        else:
            Rsteptime.append(0)
            Lstep_time = strike_df.loc[idx, 'LHSidx'] - strike_df.loc[idx,'RHSidx']
            Lsteptime.append(Lstep_time)          
    else:
        if strike_df.loc[0, 'LHSidx'] < strike_df.loc[0, 'RHSidx']:
            Rstep_time = strike_df.loc[idx, 'RHSidx'] - strike_df.loc[idx, 'LHSidx']
            Rsteptime.append(Rstep_time)
            Lstep_time = strike_df.loc[idx, 'LHSidx'] - strike_df.loc[idx - 1,'RHSidx']
            Lsteptime.append(Lstep_time)
        else:
            Rstep_time = strike_df.loc[idx, 'RHSidx'] - strike_df.loc[idx - 1, 'LHSidx']
            Rsteptime.append(Rstep_time)
            Lstep_time = strike_df.loc[idx, 'LHSidx'] - strike_df.loc[idx,'RHSidx']
            Lsteptime.append(Lstep_time)
        

strike_df['Rstep_time'] = Rsteptime
strike_df['Lstep_time'] = Lsteptime

strike_df.tail()

,LHSidx,lHSinterval,LHSval,RHSidx,rHSinterval,RHSval,Rstep_time,Lstep_time
296,29533,93,46.827446,29486,91,171.508469,46,47
297,29630,97,111.961418,29579,93,176.842056,46,51
298,29730,100,191.395111,29679,100,266.547852,49,51
299,29830,100,161.379471,29779,100,260.919281,49,51
300,29928,98,117.548874,29877,98,253.418274,47,51


In [22]:
# Get Treadmil Belt Speed
input_val = pd.read_csv('input_val.csv')
input_val = input_val.set_index('Unnamed: 0')
beltspeed = input_val.loc[Subject, 'belt_speed']
beltspeed, type(beltspeed)

(1.07, numpy.float64)

In [896]:
#Calculate Step Length

Rsteplength = []
Lsteplength = []

for idx in strike_df.index:
    if strike_df.loc[0, 'LHSidx'] < strike_df.loc[0, 'RHSidx']:
        if idx == 0:
            Rstep_length = strike_df.loc[idx, 'RHSval'] - (strike_df.loc[idx, 'LHSval'] - (beltspeed*strike_df.loc[idx, 'Rstep_time']*10))
            Rsteplength.append(Rstep_length)
            Lsteplength.append(0)
        else:
            Rstep_length = strike_df.loc[idx, 'RHSval'] - (strike_df.loc[idx, 'LHSval'] - (beltspeed*strike_df.loc[idx, 'Rstep_time']*10))
            Rsteplength.append(Rstep_length)
            Lstep_length = strike_df.loc[idx, 'LHSval'] - (strike_df.loc[idx - 1,'RHSval']-(beltspeed*strike_df.loc[idx, 'Lstep_time']*10))
            Lsteplength.append(Lstep_length)
    else:
        if idx ==0:
            Rsteplength.append(0)
            Lstep_length = strike_df.loc[idx, 'LHSval'] - (strike_df.loc[idx,'RHSval']-(beltspeed*strike_df.loc[idx, 'Lstep_time']*10))
            Lsteplength.append(Lstep_length)
        else:
            Rstep_length = strike_df.loc[idx, 'RHSval'] - (strike_df.loc[idx - 1, 'LHSval'] - (beltspeed*strike_df.loc[idx, 'Rstep_time']*10))
            Rsteplength.append(Rstep_length)
            Lstep_length = strike_df.loc[idx, 'LHSval'] - (strike_df.loc[idx,'RHSval']-(beltspeed*strike_df.loc[idx, 'Lstep_time']*10))
            Lsteplength.append(Lstep_length)
        

strike_df['Rstep_length'] = Rsteplength
strike_df['Lstep_length'] = Lsteplength

strike_df.tail()


,LHSidx,lHSinterval,LHSval,RHSidx,rHSinterval,RHSval,Rstep_time,Lstep_time,Rstep_length,Lstep_length
206,29384,156,-1299.689209,29315,169,-1286.771484,87,69,480.572217,359.682275
207,29518,134,-1371.113037,29450,135,-1323.295654,66,68,332.793555,319.382617
208,29658,140,-1346.955200,29587,137,-1360.916748,69,71,382.796289,397.361548
209,29790,132,-1324.000610,29721,134,-1268.549438,63,69,418.605762,317.148828
210,29937,147,-1289.959839,29860,139,-1266.405518,70,77,435.595092,392.245679


In [23]:
#Calculate Step Length 2
Rsteplength = []
Lsteplength = []

for idx in strike_df.index:
    if idx == 0:
        if strike_df.loc[0, 'LHSidx'] < strike_df.loc[0, 'RHSidx']:
            Rstep_length = strike_df.loc[idx, 'RHSval'] - (strike_df.loc[idx, 'LHSval'] - (beltspeed*strike_df.loc[idx, 'Rstep_time']*10))
            Rsteplength.append(Rstep_length)
            Lsteplength.append(0)
        else:
            Rsteplength.append(0)
            Lstep_length = strike_df.loc[idx, 'LHSval'] - (strike_df.loc[idx,'RHSval']-(beltspeed*strike_df.loc[idx, 'Lstep_time']*10))
            Lsteplength.append(Lstep_length)
            
    else:
        if strike_df.loc[0, 'LHSidx'] < strike_df.loc[0, 'RHSidx']:
            Rstep_length = strike_df.loc[idx, 'RHSval'] - (strike_df.loc[idx, 'LHSval'] - (beltspeed*strike_df.loc[idx, 'Rstep_time']*10))
            Rsteplength.append(Rstep_length)
            Lstep_length = strike_df.loc[idx, 'LHSval'] - (strike_df.loc[idx - 1,'RHSval']-(beltspeed*strike_df.loc[idx, 'Lstep_time']*10))
            Lsteplength.append(Lstep_length)
        else:
            Rstep_length = strike_df.loc[idx, 'RHSval'] - (strike_df.loc[idx - 1, 'LHSval'] - (beltspeed*strike_df.loc[idx, 'Rstep_time']*10))
            Rsteplength.append(Rstep_length)
            Lstep_length = strike_df.loc[idx, 'LHSval'] - (strike_df.loc[idx,'RHSval']-(beltspeed*strike_df.loc[idx, 'Lstep_time']*10))
            Lsteplength.append(Lstep_length)
        

strike_df['Rstep_length'] = Rsteplength
strike_df['Lstep_length'] = Lsteplength

strike_df.tail()

,LHSidx,lHSinterval,LHSval,RHSidx,rHSinterval,RHSval,Rstep_time,Lstep_time,Rstep_length,Lstep_length
296,29533,93,46.827446,29486,91,171.508469,46,47,595.654972,378.218977
297,29630,97,111.961418,29579,93,176.842056,46,51,622.214610,480.819362
298,29730,100,191.395111,29679,100,266.547852,49,51,678.886434,470.547259
299,29830,100,161.379471,29779,100,260.919281,49,51,593.824170,446.160190
300,29928,98,117.548874,29877,98,253.418274,47,51,594.938803,409.830600


In [309]:
#strike_df.to_csv('G02Trial02step.csv')

In [24]:

step = ['Rstep_time','Rstep_length','Lstep_time','Lstep_length']
rows = []

for col in step:
    df_attr = getattr(strike_df, col)
    row = pd.DataFrame({filename+"_Avg":[df_attr.mean()], 
                        filename+"_SD":[df_attr.std()]
                       }, index = [col])
    rows.append(row)

step_df = pd.concat(rows)
step_df

,G02Trial02A50_Avg,G02Trial02A50_SD
Rstep_time,47.774086,3.634390
Rstep_length,618.975905,49.350906
Lstep_time,51.538206,3.537801
Lstep_length,443.166298,48.540851


In [26]:
if not os.path.exists('step.csv'):
    step_df.to_csv('step.csv', index=step, mode='w')
else:
    step_df.to_csv('step.csv', index=step, mode='a')

In [246]:
#step_df.to_csv('./step.csv', index=False)
#step = open('step.csv',"w")
#step

<_io.TextIOWrapper name='step.csv' mode='w' encoding='UTF-8'>

In [248]:
#step_df2 = pd.concat([step, step_df], axis=1)
#step_df2